In [5]:
# NOTE: The way the functions are arranged 
# is a bit repetitive, you can group method in modules
# then you can from npmine.util import something

# Papers to use as example
# https://www.sciencedirect.com/science/article/pii/003194229085002W?via%3Dihub
# Download all papers Quimica Nova
# http://www.scielo.br/pdf/qn/v35n11/v35n11a34.pdf


from npmine.retrieve_doi import retrieve_doi
from npmine.retrieve_chemical_entities import retrieve_chemical_entities
from npmine.retrieve_chemical_entities import download_pdf
from npmine.retrieve_chemical_entities_from_image import retrieve_chemical_entities_from_image

import os
import json

In [2]:
# Retrieve all DOis from this journal
# NOTE: The function should contain the parameter referring to the journal
doi = retrieve_doi()

In [3]:
doi

['https://pubs.acs.org/doi/pdf/10.1021/np50001a700',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a001',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a002',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a003',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a004',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a005',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a006',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a007',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a008',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a009',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a010',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a700',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a001',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a002',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a003',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a004',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a005',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a006',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002

In [11]:
# write doi file
#with open('../data/doi_dnp.json', 'w+') as outfile:
#    json.dump(doi, outfile, indent=4)

# compress the json file
!zip -j ../data/doi_dnp.zip ../data/doi_dnp.json

  adding: doi_dnp.json (deflated 93%)


In [2]:
# uncompress the json file
#!unzip ../data/doi_dnp.zip -d ../data/

# Load the file, so you don't need to generate it
# every time
with open('../data/doi_dnp.json', 'r') as inputfile:
    doi = json.load(inputfile)

In [3]:
len(doi)

14481

In [8]:
doi[-3:]

['https://pubs.acs.org/doi/pdf/10.1021/acs.jnatprod.9b00914',
 'https://pubs.acs.org/doi/pdf/10.1021/npv082i012_1340240',
 'https://pubs.acs.org/doi/pdf/10.1021/npv082i012_1340198']

In [ ]:
# Separating the download
# from entity retrieval
# is more efficient
for url in alldoi:
    doi = link.split('/')[-1]
    download_pdf(url, '%s.pdf' % doi)

In [6]:
# We have all jnatprod files for test!
os.chdir('jnatprod/')
chemical_entities = [retrieve_chemical_entities(d) for d in doi[:5]]

In [7]:
chemical_entities

[{'np50001a700': {'oscar': [{'md5Sum': 'b81e6d19ddfacac548f900436a353648',
     'chemicalData': {}}]}},
 {'np50001a001': {'oscar': [{'md5Sum': '50ea957c9d336c0f1cbabed8d616d4fd',
     'chemicalData': {'acetone': {'name': 'acetone',
       'standardInChI': 'InChI=1S/C3H6O/c1-3(2)4/h1-2H3',
       'standardInChIKey': 'CSCPPACGZOOCGX-UHFFFAOYSA-N'},
      'dichloromethane': {'name': 'dichloromethane',
       'standardInChI': 'InChI=1S/CH2Cl2/c2-1-3/h1H2',
       'standardInChIKey': 'YMWUJEATGCHHMB-UHFFFAOYSA-N'},
      'DMSO': {'name': 'DMSO',
       'standardInChI': 'InChI=1S/C2H6OS/c1-4(2)3/h1-2H3',
       'standardInChIKey': 'IAZDPXIOMUYVGZ-UHFFFAOYSA-N'},
      'ammonia': {'name': 'ammonia',
       'standardInChI': 'InChI=1S/H3N/h1H3',
       'standardInChIKey': 'QGZKDVFQNNGYKY-UHFFFAOYSA-N'},
      'ammonium hydroxide': {'name': 'ammonium hydroxide',
       'standardInChI': 'InChI=1S/H3N.H2O/h1H3;1H2',
       'standardInChIKey': 'VHUUQVKOLVNVRT-UHFFFAOYSA-N'},
      'BISBENZYLISOQUIN

In [11]:
retrieve_chemical_entities_from_image(doi[0])

{'np50001a700': {'osra': []}}